In [ ]:
#Bargainng
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools
import pickle
import statsmodels.api as sm
import seaborn as sns
#from binsreg import binsreg
from binsreg import *
from plotnine import *
from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20
T = 20


import warnings
warnings.filterwarnings("ignore")


In [ ]:
name = 'model_LC'
model_name = "model_LC"



do_distr_yerror = "none"
do_yerror = 0
name_error = f'{do_distr_yerror}{do_yerror}std_Yerror'
N = 3000 
NN = 9000
gender = "woman"

specs = {}


specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':1}}})
#specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':2}}})
#specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'simN': N,  'threads':threads,'bargaining':0}}})

# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    
    # setup model
    model = LimitedCommitmentModelClass(name=name,par=spec['par'])
    model.spec = spec

    compile_now = True if do_compile & (m==0) else False
    model.link_to_cpp(force_compile=compile_now)
    

# setup model
#model = LimitedCommitmentModelClass(name=name)    
#model.link_to_cpp(force_compile=True)


#load par and sol
pickle_file = f'output/{name}_sol.pickle'

with open(pickle_file, 'rb') as f:
    model.sol = pickle.load(f)

pickle_file = f'output/{name}_par.pickle'

with open(pickle_file, 'rb') as f:
    model.par = pickle.load(f)
    model.par.simN = N

In [ ]:
data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror )
data = Estimate.create_variable(data, model.par, print_aux_reg = True)
#data = Estimate.aux_est(data,model.par)

data2 = data


summary = data2[['delta_log_barganing']].describe()



_, wald_FC_w , wald_NC_w,  _,_,_ = Estimate.main_est(data2,gender= "w", do_estimate_wage = "est_omega", print_reg = True, shadow_value_simple=2,part_earning_simple = 2, control_cons = 2, do_control_love = True , do_true_barg = False, control_select = False)


data2 =data2[data2['omega_res_w'] < 0.1]
data2 =data2[data2['omega_res_w'] > -0.1]
data2 =data2[data2['omega_res_m'] < 0.1]
data2 =data2[data2['omega_res_m'] > -0.1]

data2 =data2[data2['omega_res_w_l'] < 0.1]
data2 =data2[data2['omega_res_w_l'] > -0.1]
data2 =data2[data2['omega_res_m_l'] < 0.1]
data2 =data2[data2['omega_res_m_l'] > -0.1]
data2 =data2[data2['omega_res_w_l2'] < 0.1]
data2 =data2[data2['omega_res_w_l2'] > -0.1]
data2 =data2[data2['omega_res_m_l2'] < 0.1]
data2 =data2[data2['omega_res_m_l2'] > -0.1]

summary2 = data2[['delta_log_barganing']].describe()

_, wald_FC_w , wald_NC_w,  _,_,_ = Estimate.main_est(data2,gender= "w", do_estimate_wage = "est_omega", print_reg = True, shadow_value_simple=2,part_earning_simple = 2, control_cons = 2, do_control_love = True , do_true_barg = False, control_select = False)


data2['delta_log_barganing'] = data2['delta_log_barganing'].mask(data2['delta_log_barganing'].between(-0.001,0.001))

_, wald_FC_w , wald_NC_w,  _,_,_ = Estimate.main_est(data2,gender= "w", do_estimate_wage = "est_omega", print_reg = True, shadow_value_simple=2,part_earning_simple = 2, control_cons = 2, do_control_love = True , do_true_barg = False, control_select = False)


In [ ]:
#CORRELATION

data_regress = data
data_regress = data_regress[data_regress['delta_log_barganing_l']< 1.0 ]
data_regress = data_regress[data_regress['delta_log_barganing_l'] > - 1.0 ]
data_regress = data_regress[data_regress['delta_log_barganing']< 1.0 ]
data_regress = data_regress[data_regress['delta_log_barganing'] > -1.0 ]

list =[ 'wage_shock','wage_shock_j','BMI','BMI_j','delta_log_wealth','delta_log_Love','delta_log_barganing_l']

list =['delta_log_barganing_l']

for i in list:
    print(f'{i}')

    joint_plot = sns.jointplot(data_regress, x=f'{i}', y=f'delta_log_barganing')
    #joint_plot.fig.savefig(f'output/bargaining_x{i}_plot{model_name}_{gender}.png')
    #binspos='es' #like this equal spread does not work properly
    est = binsreg(data_regress['delta_log_barganing'],data_regress[f'{i}'], nbins=100,polyreg=1, noplot=False)
  





In [ ]:
p = 1 #p=1 normale, p=2 2 lag, p = 3, 2 lag inkl initiale , p = 4 gør inkl bargainging


def regress_barg(data,  p=1, do_true_barg = False, koen = "w", print_result = False, do_fig_est = False):
    gender = f'{koen}'
    #Find the gender and the gender of the spouse
    if gender == "w":
        spouse = "m"
    elif gender =='m':
        spouse = "w"
    if do_true_barg: 
       


        data['wage_shock']=data[f'delta_log_omega_{gender}']*data[f'omega_{gender}_l']
        data['wage_shock_j']=data[f'delta_log_omega_{spouse}']*data[f'omega_{spouse}_l']
        data['wage_shock_l']=data[f'delta_log_omega_{gender}_l']*data[f'omega_{gender}_l2']
        data['wage_shock_j_l']=data[f'delta_log_omega_{spouse}_l']*data[f'omega_{spouse}_l2']
        data['wage_shock_l']=data[f'delta_log_omega_{gender}_l2']*data[f'omega_{gender}_l3']
        data['wage_shock_j_l']=data[f'delta_log_omega_{spouse}_l2']*data[f'omega_{spouse}_l3']




        
        data['BMI'] = data[f'delta_log_BMI_{gender}']*data[f'BMI_{gender}_l']*data[f'barg_l_inv']
        data['BMI_j'] = data[f'delta_log_BMI_{spouse}']*data[f'BMI_{spouse}_l']*data[f'barg_l_inv']
        data['BMI_l'] = data[f'delta_log_BMI_{gender}_l']*data[f'BMI_{gender}_l2']*data[f'barg_l_inv']
        data['BMI_j_l'] = data[f'delta_log_BMI_{spouse}_l']*data[f'BMI_{spouse}_l2']*data[f'barg_l_inv']
        data['BMI_l2'] = data[f'delta_log_BMI_{gender}_l2']*data[f'BMI_{gender}_l3']*data[f'barg_l_inv']
        data['BMI_j_l2'] = data[f'delta_log_BMI_{spouse}_l2']*data[f'BMI_{spouse}_l3']*data[f'barg_l_inv']



        data['delta_log_wealth'] = data[f'delta_log_wealth']*data[f'wealth_l']*data[f'barg_l_inv']
        data['delta_log_wealth_l'] = data[f'delta_log_wealth_l']*data[f'wealth_l2']*data[f'barg_l_inv']
        data['delta_log_wealth_l2'] = data[f'delta_log_wealth_l2']*data[f'wealth_l3']*data[f'barg_l_inv']

        data['delta_log_Love'] = data[f'delta_log_Love']*data[f'Love_l']*data[f'barg_l_inv']
        data['delta_log_Love_l'] = data[f'delta_log_Love_l']*data[f'Love_l2']*data[f'barg_l_inv']
        data['delta_log_Love_l2'] = data[f'delta_log_Love_l2']*data[f'Love_l3']*data[f'barg_l_inv']

        data['delta_log_barganing'] = data[f'delta_log_barganing']
        data['delta_log_barganing_l'] = data[f'delta_log_barganing_l']*data[f'barganing_l2']*data[f'barg_l_inv']
        data['delta_log_barganing_l2'] = data[f'delta_log_barganing_l2']*data[f'barganing_l3']*data[f'barg_l_inv']
        data['delta_log_barganing_l3'] = data[f'delta_log_barganing_l3']*data[f'barganing_l4']*data[f'barg_l_inv']
    else: 
            
        data['wage_shock']=data[f'omega_res_{gender}']
        data['wage_shock_l']=data[f'omega_res_{gender}_l']
        data['wage_shock_l2']=data[f'omega_res_{gender}_l2']
        data['wage_shock_j']=data[f'omega_res_{spouse}']
        data['wage_shock_j_l']=data[f'omega_res_{spouse}_l']
        data['wage_shock_j_l2']=data[f'omega_res_{spouse}_l2']
        
        data['wage_shock']=data[f'delta_log_omega_{gender}']
        data['wage_shock_j']=data[f'delta_log_omega_{spouse}']
        data['wage_shock_l']=data[f'delta_log_omega_{gender}_l']
        data['wage_shock_j_l']=data[f'delta_log_omega_{spouse}_l']
        data['wage_shock_l']=data[f'delta_log_omega_{gender}_l2']
        data['wage_shock_j_l']=data[f'delta_log_omega_{spouse}_l2']


        
        data['BMI'] = data[f'delta_log_BMI_{gender}']
        data['BMI_l'] = data[f'delta_log_BMI_{gender}_l']
        data['BMI_l2'] = data[f'delta_log_BMI_{gender}_l2']
        data['BMI_j'] = data[f'delta_log_BMI_{spouse}']
        data['BMI_j_l'] = data[f'delta_log_BMI_{spouse}_l']
        data['BMI_j_l2'] = data[f'delta_log_BMI_{spouse}_l2']



        

    if p == 1:
        data_regress = data[['delta_log_barganing', 'wage_shock','wage_shock_j','BMI','BMI_j','delta_log_wealth','delta_log_barganing_l','t']]
        list = ['wage_shock','wage_shock_j','BMI','BMI_j', 'delta_log_barganing_l']
        list = [ 'delta_log_barganing_l',]


    if p ==2:
        data_regress = data[['delta_log_barganing', 'wage_shock','wage_shock_j','BMI','BMI_j','delta_log_wealth','delta_log_Love','delta_log_barganing_l','t']]
        
       
        list = ['wage_shock','wage_shock_j','BMI','BMI_j','delta_log_barganing_l']
        list = [ 'delta_log_barganing_l',]
    if p ==3:
        data_regress = data[['delta_log_barganing', 'wage_shock','wage_shock_l','wage_shock_l2','wage_shock_j','wage_shock_j_l','wage_shock_j_l2','BMI','BMI_l','BMI_l2','BMI_j','BMI_j_l','BMI_j_l2','delta_log_wealth','delta_log_wealth_l','delta_log_wealth_l2','init_barg','t']]
        
        #PREPARE T
        X_t=pd.get_dummies(data_regress[['init_barg']], columns = ['init_barg'], prefix = ['D_init_barg'], dtype = float) 
        X_t = X_t.drop(columns = ['D_init_barg_1']) #drop reference 
        
        data_regress = data_regress.drop(columns = ['init_barg'])
        
        data_regress = pd.concat([data_regress,  X_t], axis=1 )
        
        list = ['wage_shock','wage_shock_l','wage_shock_l2','wage_shock_j','wage_shock_j_l','wage_shock_j_l2','BMI','BMI_l','BMI_l2','BMI_j','BMI_j_l','BMI_j_l2']


    if p ==4:
        data_regress = data[['delta_log_barganing', 'wage_shock','wage_shock_l','wage_shock_l2','wage_shock_j','wage_shock_j_l','wage_shock_j_l2','BMI','BMI_l','BMI_l2','BMI_j','BMI_j_l','BMI_j_l2','delta_log_wealth','delta_log_wealth_l','delta_log_wealth_l2','delta_log_Love','delta_log_Love_l','delta_log_Love_l2','t','delta_log_barganing_l3']]
        
        list = ['wage_shock','wage_shock_l','wage_shock_l2','wage_shock_j','wage_shock_j_l','wage_shock_j_l2','BMI','BMI_l','BMI_l2','BMI_j','BMI_j_l','BMI_j_l2','delta_log_barganing_l3']






    #DROP NAN
    data_regress = data_regress.dropna() 

    
    X_t=pd.get_dummies(data_regress[['t']], columns = ['t'], prefix = ['D_t'], dtype = float) 
    X_t = X_t.drop(columns = ['D_t_13']) #drop reference cat


        


    df = data_regress.drop(columns = ['delta_log_barganing','t'])
    
    #REGRESS

    y  =  data_regress[['delta_log_barganing']]
       
    x1 = pd.concat([df,  X_t ], axis=1 )
    x1 = sm.add_constant(x1)  

    result2= sm.OLS(y,x1).fit()
    if print_result:
        print(result2.summary())
    res = result2.resid
    
    predict = result2.predict()
    data_regress['res']  = res
    data_regress['predict']  = predict
    if do_fig_est:
        print('residual plot')
        plt.scatter(predict,res)
        plt.ylabel("Residual Delta log mu")
        plt.xlabel(f'Predicted value Delta log mu')

        plt.savefig(f'output/barg_res_predict_plot{model_name}_{gender}.png')
        plt.show()
        joint_plot = sns.jointplot(data_regress, x=predict, y=res)
        est = binsreg(res,predict, nbins=100,polyreg=1, noplot=False) 


        #print('res against y')
        #plt.scatter(data_regress[['delta_log_barganing']],res)
        #plt.show()

    if do_fig_est:

        for i in list :
            

            xlabel = f'{i}'
            if i == "wage_shock":
                xlabel = "Shock to wage"
            elif i == "wage_shock_j":
                xlabel = "Shock to partners wage"
            elif i == "delta_log_barganing_l":
                xlabel = "Change in barganing weigth, lag"
            x = x1
            y2 = x[f'{i}']
            x = x.drop(columns = [f'{i}'])
            
            #x = x.drop(columns = list_drop)
            #x = x.T.drop_duplicates().T #drop duplicates
            x = sm.add_constant(x)  
            
            result= sm.OLS(y,x).fit()
            res_y = result.resid
            data_regress[f'res_y'] = res_y
            
            result= sm.OLS(y2,x).fit()
            res_x = result.resid
            data_regress[f'res_x']=res_x


            
            print(f'{i}')
            joint_plot = sns.jointplot(data_regress, x='res_x', y='res_y')
            est = binsreg(res_y,res_x, nbins=100,polyreg=1, noplot=False) 
            
           
    coef_dict = {}
    coef_dict['wage_shock_j'] = result2.params['wage_shock_j']
    coef_dict['wage_shock'] = result2.params['wage_shock']
    coef_dict['BMI'] = result2.params['BMI']
    coef_dict['BMI_j'] = result2.params['BMI_j']

    coef_dict['delta_log_barganing_l'] = result2.params['delta_log_barganing_l']
    return coef_dict


In [ ]:

coef = {}

no_sim = 100


for i in range(no_sim):
    print_result = False

    data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror )
    data = Estimate.create_variable(data, model.par, print_aux_reg = False)
    

    data2 = data
    
    #outliers 
    data2 = data2[data2['delta_log_barganing'] < 1.0]
    data2 = data2[data2['delta_log_barganing'] > - 1.0]

    data2 = data2[data2['delta_log_barganing_l'] < 1.0]
    data2 = data2[data2['delta_log_barganing_l'] > - 1.0]
    for gen in ['w','m']:
        #deres
        
        p = 1 
        j = False
        ij = int(j)
        k = 0 #non restrict sample
        gender = f'{gen}'
        c = regress_barg(data2, p=p, do_true_barg =j, koen = gender, print_result =print_result, do_fig_est = False)

        coef[f'est{p}_true_barg{ij}_restr_sample{k}_sim{i}_gender{gen}'] =  c


        p = 2 
        j = False
        ij = int(j)
        k = 0 #non restrict sample
        gender = f'{gen}'
        c = regress_barg(data2, p=p, do_true_barg =j, koen = gender, print_result =print_result, do_fig_est = False)

        coef[f'est{p}_true_barg{ij}_restr_sample{k}_sim{i}_gender{gen}'] =  c
        
        #sample selection
        # 
        k = 1
        data2 = data

        #outliers 
        data2 = data2[data2['delta_log_barganing'] < 1.0]
        data2 = data2[data2['delta_log_barganing'] > - 1.0]

        data2 = data2[data2['delta_log_barganing_l'] < 1.0]
        data2 = data2[data2['delta_log_barganing_l'] > - 1.0]
        data2 =data2[data2['omega_res_w'] < 0.1]
        data2 =data2[data2['omega_res_w'] > -0.1]
        data2 =data2[data2['omega_res_m'] < 0.1]
        data2 =data2[data2['omega_res_m'] > -0.1]



        data2 = data2[data2['delta_log_barganing_l'] < 0.1]
        data2 = data2[data2['delta_log_barganing_l'] > - 0.1]


        
        #p = 1 #p=1 normale, p=2 selection, p = 3, 2 lag inkl initiale , p = 4 gør inkl bargainging
        p = 1
        j = False
        ij = int(j)
        c =regress_barg(data2, p=p, do_true_barg =j, koen = gender, print_result =print_result, do_fig_est = False)

        coef[f'est{p}_true_barg{ij}_restr_sample{k}_sim{i}_gender{gen}'] =  c


        p = 2
        j = False
        ij = int(j)
        c =regress_barg(data2, p=p, do_true_barg =j, koen = gender, print_result =print_result, do_fig_est = False)

        coef[f'est{p}_true_barg{ij}_restr_sample{k}_sim{i}_gender{gen}'] =  c
        
        #korrekt bargaining
    
        
        k = 2
        data2['delta_log_barganing'] = data2['delta_log_barganing'].mask(data2['delta_log_barganing'].between(-0.001,0.001))
        p = 1
        c =regress_barg(data2, p=p, do_true_barg =j, koen = gender, print_result =print_result, do_fig_est = False)

        coef[f'est{p}_true_barg{ij}_restr_sample{k}_sim{i}_gender{gen}'] =  c

        p = 2
        c =regress_barg(data2, p=p, do_true_barg =j, koen = gender, print_result =print_result, do_fig_est = False)

        coef[f'est{p}_true_barg{ij}_restr_sample{k}_sim{i}_gender{gen}'] =  c

     




In [ ]:
print(f'{model_name}')
pickle_file = f'output/barg_TRUE{name}_{NN}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef,f)

k = 1
p = 1
ij = 0
for i in range(10):
    print(coef[f'est{p}_true_barg{ij}_restr_sample{k}_sim{i}_genderw'] )



In [ ]:
c =regress_barg(data2, p=1, do_true_barg = False, koen = "w", print_result =True, do_fig_est = True)



In [ ]:
data2 = data
c =regress_barg(data2, p=1, do_true_barg = False, koen = "w", print_result =True, do_fig_est = False)
#12000
data2 =data2[data2['omega_res_w'] < 0.1]
data2 =data2[data2['omega_res_w'] > -0.1]
data2 =data2[data2['omega_res_m'] < 0.1]
data2 =data2[data2['omega_res_m'] > -0.1]


data2 =data2[data2['omega_res_w_l'] < 0.1]
data2 =data2[data2['omega_res_w_l'] > -0.1]
data2 =data2[data2['omega_res_m_l'] < 0.1]
data2 =data2[data2['omega_res_m_l'] > -0.1]


data2 =data2[data2['omega_res_w_l2'] < 0.1]
data2 =data2[data2['omega_res_w_l2'] > -0.1]
data2 =data2[data2['omega_res_m_l2'] < 0.1]
data2 =data2[data2['omega_res_m_l2'] > -0.1]



#data2 =data2[data2['delta_log_barganing'] != 0.0]
#data2 =data2[data2['delta_log_barganing']  != 0.0]

c =regress_barg(data2, p=1, do_true_barg = False, koen = "w", print_result =True, do_fig_est = False)



data2['delta_log_barganing'] = data2['delta_log_barganing'].mask(data2['delta_log_barganing'].between(-0.001,0.001))

#data2 =data2[data2['delta_log_barganing'] != 0.0]
#data2 =data2[data2['delta_log_barganing']  != 0.0]

c =regress_barg(data2, p=1, do_true_barg = False, koen = "w", print_result =True, do_fig_est = False)